In [ ]:
#Importing Libraries
import pandas as pd
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

In [ ]:
df = pd.read_csv("amazon_n1.csv") #Reading in Saved Dataset
#Filling NA Values in Subtitle and Series with None
df['subtitle'].fillna(value = 'None' , inplace = True)
df['Series'].fillna(value = 'None' , inplace = True) 

df = df[df['title'] != 'No Title'] #Removing Isbns with No Titles

In [ ]:
#Creating a column that places prefix , subtitle and series together in order to compare it to the title given by the retail site 
title = []
for i , j , k in zip(df['Title with Prefix'], df['subtitle'] , df['Series']):
    
    if (j == 'None') & (k == 'None'):
        title.append(str(i))
    elif (j == 'None') & (k == 'No Series'):
        title.append(str(i))
    elif (j == 'None') & (k == 'NONE'):
        title.append(str(i))
    
    
    elif (j == 'None') &(k != 'None'):
        title.append(str(i) + ':' + str(k))
        
    elif (j == 'None') &(k != 'No Series'):
        title.append(str(i) + ':' + str(k))
        
    elif (j == 'None') &(k != 'NONE'):
        title.append(str(i) + ':' + str(k))
        
    elif (k == 'None') &(j != 'None'):
        title.append(str(i) + ':' + str(j))
    elif (k == 'NONE') & (j != 'None'):
        title.append(str(i) + ':' + str(j))
    elif (k == "No Series") & (j != 'None'):
        title.append(str(i) + ':' + str(j))
    
    else: 
        title.append(str(i) + ':' + str(j) + '(' + str(k) + ')')
        
        
df['Full_Title'] = title #adding column to data 

In [ ]:
eans = list(df['ean_13'].unique()) #Getting a list of titles 
#Getting a list of title , prefix , subtitle ,  and the full title from the column we added
title = [] 
prefix = [] 
subtitle = [] 
fullTitle = []
for i in eans: 
    d = df.loc[df['ean_13'] == i]
    title.append(list(d['title'].unique())[0])
    prefix.append(list(d['Title with Prefix'].unique())[0])
    subtitle.append(list(d['subtitle'].unique())[0])
    fullTitle.append(list(d['Full_Title'].unique())[0])

In [ ]:
#Comparing title from retail site to title from publisher and getting a ratio 
ratio = [] 
for i , j in zip(title, fullTitle): 
    ratio.append(fuzz.partial_ratio(i, j))
    
    
#Checking if prefix is in the title
prefixcheck = [] 
for t , p in zip(title, prefix):
    prefixcheck.append(p in t)
    
    
#Checking if subtitle is in the title
subcheck = [] 
for t , s in zip(title, subtitle):
    subcheck.append(s in t)

In [ ]:
#Tagging titles based on ratio 
label = [] 
for i in ratio: 
    if 26 <= i <= 50: 
        label.append("Urgent")
    elif 50 <= i <=  70: 
        label.append("Okay")
    elif 70 <=   i <= 85:
        label.append("Moderate")
    else:
        label.append("Great")

In [ ]:
#Getting avg price information for each isbn
avg_listp = [] 
avg_salep = []
avg_newp = [] 
avg_usdp = [] 
for i in eans: 
    d = df.loc[df['ean_13'] == i]
    avg_listp.append(d['list_price'].mean())
    avg_salep.append(d['for_sale_price'].mean())
    avg_newp.append(d['new_price'].mean())
    avg_usdp.append(d['USD MSR Price'].mean())
    

In [ ]:
#Putting data together 
data_1 = pd.DataFrame({'ean_13': eans, 
                    'Title': title, 
                     'FullTitle_publisher': fullTitle, 
                    'Prefix': prefix, 
                    'Subtitle': subtitle, 
                       'Avg_ListPrice': avg_listp, 
                       'Avg_ForSalePrice': avg_salep, 
                       'Avg_NewPrice': avg_newp, 
                       'Avg_USDPrice': avg_usdp,
                    'PrefixCheck': prefixcheck, 
                    'SubtitleCheck': subcheck, 
                    'Similarity_Ratio': ratio,
                       'Title_Label': label})


In [ ]:
data_1.to_csv("Title_Check_amazon.csv", index = False) #Saving Data 